In [ ]:
# https://www.tensorflow.org/datasets/keras_example

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import logging, os
import pandas as pd

logging.basicConfig(level=logging.INFO)
# logging.basicConfig(level=logging.DEBUG)

BASE_PATH = "../../../../../local_data/practice/tfds/"
DATA_PATH = "../../../../../local_data/tfds/"
OUTPUT_PATH = BASE_PATH+"mnist_simple_01/"
os.system("mkdir -p " + OUTPUT_PATH)


(train_ds, test_ds), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(f"Number of training samples: {train_ds.cardinality()}")
print(f"Number of test samples: {test_ds.cardinality()}")

# plt.figure(figsize=(10, 10))
# for i, (image, label) in enumerate(train_ds.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(image)
#     plt.title(int(label))
#     plt.axis("off")

2025-07-07 21:29:32.571897: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 21:29:32.604707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:absl:Load dataset info from /home/tmeng12/tensorflow_datasets/mnist/3.0.1
INFO:absl:Fields info.[citation, splits, supervised_keys, module_name] from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset mnist (/home/tmeng12/tensorflow_datasets/mnist/3.0.1)
INFO:absl:Constructing tf.data.Dataset mnist for split ['trai

Number of training samples: 60000
Number of test samples: 10000


In [2]:
c1=tfds.as_dataframe(train_ds.take(10), ds_info)
col1 = c1['label']
# col1 = pd.DataFrame(train_ds, columns=["image","train_ds"])
print(col1)

0    4
1    1
2    0
3    7
4    8
5    1
6    2
7    7
8    1
9    6
Name: label, dtype: int64


2025-07-07 21:29:34.899715: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [3]:
batch_size=16

In [4]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

train_ds = train_ds.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.cache()
# train_ds = train_ds.shuffle(ds_info.splits['train'].num_examples)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

In [5]:
test_ds = test_ds.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(batch_size, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    train_ds,
    epochs=1,
    validation_data=test_ds,
)

3750/3750 [==============================] - 6s 2ms/step - loss: 0.3957 - sparse_categorical_accuracy: 0.8884 - val_loss: 0.2561 - val_sparse_categorical_accuracy: 0.9271


In [7]:
c1=tfds.as_dataframe(train_ds.take(5000), ds_info)
col1 = c1['label']
print(col1.shape)

pred = model.predict(train_ds)
# print("pred\n",pred)
# col2 = pd.DataFrame(pred[0:20], columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
col2 = pd.DataFrame(pred, columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
# print(col2)

compare = pd.concat([col1, col2], axis=1)
print(compare)
compare.to_csv(OUTPUT_PATH + "pred_train.csv", index=False)

(3750,)
3750/3750 [==============================] - 3s 670us/step
                                                  label        p0         p1  \
0      [4, 1, 0, 7, 8, 1, 2, 7, 1, 6, 6, 4, 7, 7, 3, 3] -7.974104  -1.874438   
1      [7, 9, 9, 1, 0, 6, 6, 9, 9, 4, 8, 9, 4, 7, 3, 3] -7.499573   4.179980   
2      [0, 9, 4, 9, 0, 6, 8, 4, 7, 2, 6, 0, 3, 1, 1, 7]  6.075268 -10.879297   
3      [2, 4, 4, 6, 5, 1, 9, 3, 2, 4, 3, 4, 4, 7, 5, 8] -3.154041 -13.878533   
4      [1, 1, 4, 1, 5, 3, 5, 8, 4, 1, 1, 4, 5, 3, 2, 4] -2.916331  -8.122678   
...                                                 ...       ...        ...   
59995                                               NaN -7.877224 -12.248287   
59996                                               NaN -9.165756   4.454875   
59997                                               NaN  0.808035 -11.091747   
59998                                               NaN -8.930107   5.770471   
59999                                               N

In [8]:
pred = model.predict(test_ds)
# print("pred\n",pred)
# col2 = pd.DataFrame(pred[0:20], columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
col2 = pd.DataFrame(pred, columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
# print(col2)

compare = pd.concat([col1, col2], axis=1)
print(compare)
compare.to_csv(OUTPUT_PATH + "pred_test.csv", index=False)

625/625 [==============================] - 1s 867us/step
                                                 label         p0         p1  \
0     [4, 1, 0, 7, 8, 1, 2, 7, 1, 6, 6, 4, 7, 7, 3, 3]  -3.265374  -4.515430   
1     [7, 9, 9, 1, 0, 6, 6, 9, 9, 4, 8, 9, 4, 7, 3, 3]  10.253217 -17.563599   
2     [0, 9, 4, 9, 0, 6, 8, 4, 7, 2, 6, 0, 3, 1, 1, 7]  -7.998972 -12.060715   
3     [2, 4, 4, 6, 5, 1, 9, 3, 2, 4, 3, 4, 4, 7, 5, 8]  -4.729486   0.177743   
4     [1, 1, 4, 1, 5, 3, 5, 8, 4, 1, 1, 4, 5, 3, 2, 4]  -4.226891 -12.413978   
...                                                ...        ...        ...   
9995                                               NaN  -1.599782  -7.026804   
9996                                               NaN  -9.002808  -8.298120   
9997                                               NaN  -4.201391  -8.996775   
9998                                               NaN   9.253925 -12.211276   
9999                                               NaN  -2.3968